Install Dependencies

In [1]:
# Install Dependencies
!pip install tensorflow
!pip install matplotlib
!pip install tqdm


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\tejas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\tejas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\tejas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Extract Dataset

In [ ]:
# Local Dataset Path
DATASET = "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\dataset.zip"

# Extract Dataset with Progress Bar
import zipfile
from tqdm import tqdm  # Import tqdm for the progress bar

# Replace "/content/drive/MyDrive/deepfakeimgdetection/dataset.zip" with the local path
with zipfile.ZipFile(DATASET, 'r') as zip_ref:
    # Get the total number of files in the zip archive for progress tracking
    num_files = len(zip_ref.namelist())
    
    # Use tqdm to create a progress bar
    with tqdm(total=num_files, desc='Extracting', unit=' files') as pbar:
        for file in zip_ref.namelist():
            zip_ref.extract(file, "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection")
            pbar.update(1)  # Update progress bar


Verify Dataset

In [ ]:
import os

# Local Dataset Path
data_dir = "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\Dataset"

# Local subfolders
subfolders = ["Fake", "Real"]

def setup_directory_and_count_files(directory, subfolders):
    total_files = 0
    for subfolder in subfolders:
        subfolder_path = os.path.join(directory, subfolder)
        total_files += sum([1 for file in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, file))])
    return total_files

train_dir = os.path.join(data_dir, "Train")
test_dir = os.path.join(data_dir, "Test")
validation_dir = os.path.join(data_dir, "Validation")

total_train_files = setup_directory_and_count_files(train_dir, subfolders)
total_test_files = setup_directory_and_count_files(test_dir, subfolders)
total_validation_files = setup_directory_and_count_files(validation_dir, subfolders)

total = total_train_files + total_test_files + total_validation_files
train_perc = (total_train_files / total) * 100
test_perc = (total_test_files / total) * 100
valid_perc = (total_validation_files / total) * 100
print("Total Train Files:", total_train_files)
print("Total Test Files:", total_test_files)
print("Total Validation Files:", total_validation_files)
print(f"Train Data Percentage: {train_perc:.2f}%")
print(f"Test Data Percentage: {test_perc:.2f}%")
print(f"Validation Data Percentage: {valid_perc:.2f}%")

Model Training

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

os.environ['CUDA_VISIBLE_DEVICES'] = '0'


data_dir = "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\Dataset"  

train_dir = os.path.join(data_dir, "Train")
test_dir = os.path.join(data_dir, "Test")
validation_dir = os.path.join(data_dir, "Validation")
subfolders = ["Fake", "Real"]

def normalize_image(image, labels):
    image = tf.cast(image, tf.float32) / 255.0
    return image, labels

BATCH_SIZE = 32
IMG_SIZE = (256, 256)

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
).map(normalize_image).prefetch(buffer_size=tf.data.AUTOTUNE)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
).map(normalize_image).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
).map(normalize_image).prefetch(buffer_size=tf.data.AUTOTUNE)

model_CNN = Sequential([
    Conv2D(filters=8, kernel_size=3, padding='same', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    Activation('relu'),

    Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=128, kernel_size=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    Flatten(),
    tf.keras.layers.Dropout(0.3),
    Dense(units=64, activation='relu'),
    Dense(units=20, activation='relu'),
    Dense(units=2, activation='softmax')
])

model_CNN.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

epoch_num = 50  # Define the number of epochs
hist = model_CNN.fit(train_data,
                    epochs=epoch_num,
                    validation_data=validation_data,
                    validation_steps=int(0.5 * len(validation_data))
                    )


Save the Trained Model

In [ ]:
# @title Save the Trained Model
svpth = "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\Model"
# @markdown Save Model As:
model_name = "dfimgdet50ep" # @param {type:"string"}
modelwext = model_name + ".h"
save_path = svpth + modelwext
model_CNN.save(save_path)

Load a Model

In [2]:
# @title Load a Model
from tensorflow.keras.models import load_model
# @markdown Model Path:
model_path = "C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\Model\\model_checkpoint_10.h5" # @param {type:"string"}
loaded_model = load_model(model_path)

Upload an Image

In [4]:
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load your pre-trained model
loaded_model = load_model('C:\\Users\\tejas\\Documents\\COLLEGE\\SEMESTER 5\\EDI_DeepFake_ImageDetection\\Model\\model_checkpoint_10.h5')  # Replace with the path to your model

# Function to classify an image
def classify_image(image_path, threshold=0.5):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions using the loaded model
    predictions = loaded_model.predict(img_array)

    # Determine the classification result based on the threshold
    if predictions[0][0] >= threshold:
        result = "The uploaded image is a deepfake."
    else:
        result = "The uploaded image is not a deepfake."

    return result

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename(title="Select an Image File")
    if file_path:
        classification_result = classify_image(file_path, threshold=0.5)
        result_label.config(text=classification_result)

# Create a tkinter window
window = tk.Tk()
window.title("Deepfake Image Classification")

# Create and configure a button for file upload
upload_button = tk.Button(window, text="Upload Image", command=upload_file)
upload_button.pack(pady=20)

# Create a label to display the classification result
result_label = tk.Label(window, text="", font=("Helvetica", 16))
result_label.pack()

# Start the tkinter main loop
window.mainloop()


1/1 [==============================] - 0s 48ms/step
